In [ ]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 # Overfitting and Regularization

In the realm of predictive modeling, the pursuit of creating a model that perfectly fits the training data can inadvertently lead to a phenomenon known as overfitting. Overfitting occurs when a model becomes excessively complex, capturing not only the genuine patterns within the data but also the noise and random fluctuations present in the training set. 

The hyper-adaptation to the training data renders the model less capable of generalizing to new, unseen data, as it effectively memorizes the training examples rather than discerning meaningful relationships. As a result, an overfitted model may exhibit impressive performance on the training data but performs poorly when faced with real-world scenarios. The delicate balance between capturing essential patterns and avoiding the trap of overfitting underscores the importance of techniques like regularization, which aim to ensure model generalization by restraining excessive complexity.

There are three Regularization techniques we will be dealing with, all of which use the idea of penalizing terms to tackle overfitting.

But before we go any further, let's have a look at the data.

**Data Dictionary**

* Product_ID:- Unique identifier for each electronic device. <br>
* Product_Weight:- Weight of the device  <br>
* Energy_Efficiency:- Energy efficiency rating (e.g., "Energy Efficient", "Standard"). <br>
* Visibility_Index:- The % of the total display area of all products in a store allocated to the particular product <br>
* Product_Category:- Category of the electronic device (e.g., "Laptops", "Smartphones"). <br>
* Product_Price:- Price of the device in dollars<br>
* Store_ID:- Unique identifier for the store.<br>
* Store_Established_Year:- Number of years since the store was established.<br>
* Store_Size:- Size classification of the store (e.g., "Small", "Medium").<br>
* Location_Type:- The type of city in which the store is located<br>
* Store_Type:- Type of store (e.g., "Electronics Store Type1").<br>
* Sales_Performance:- Sales of the electronic device in the particular store. This is the outcome variable to be predicted.


## Visualizing the data

Let's visualize this all in the form of a Data Frame

In [3]:
data <- read.csv("/kaggle/input/2b-data/2b-data.csv")
head(data)


Product_ID Product_Weight Energy_Efficiency Visibility_Index Product_Category
1 FDA15       9.300         Energy Efficient  0.01604730       Laptops         
2 DRC01       5.920         Standard          0.01927822       Smartphones     
3 FDN15      17.500         Energy Efficient  0.01676007       Tablets         
4 FDX07      19.200         Standard          0.06613203       Monitors        
5 NCD19       8.930         Energy Efficient  0.06613203       Refrigerators   
6 FDP36      10.395         Standard          0.06613203       Microwaves      
  Product_Price Store_ID Store_Established_Year Store_Size Location_Type
1 249.8092      OUT049   14                     Medium     Tier 1       
2  48.2692      OUT018    4                     Medium     Tier 3       
3 141.6180      OUT049   14                     Medium     Tier 1       
4 182.0950      OUT010   15                     Small      Tier 3       
5  53.8614      OUT013   26                     High       Tier 3       
6  51.4008      OUT018    4                     Medium     Tier 3       
  Store_Type               Sales_Performance
1 Electronics Store Type1  3735.1380        
2 Electronics Store Type2   443.4228        
3 Electronics Store Type1  2097.2700        
4 Retail Electronics Store  732.3800        
5 Electronics Store Type1   994.7052        
6 Electronics Store Type2   556.6088

## Ridge Regression
Ridge regression is a linear regression technique that incorporates L2 regularization to address issues in predictive modeling (overfitting, multi-colinearity etc).
Linear regression, aims to minimize the sum of squared residuals whereas Ridge regression introduces a penalty term proportional to the square of the magnitude of the coefficients. This penalty, controlled by a hyperparameter (often denoted as lambda), discourages large coefficient values, effectively constraining the model’s complexity, enhancing its stability and generalization performance.


**1)** Perform Ridge Regression on the training data and compare the predictions with the test data to check for the fit of the model. (Hint: Use the glmnet library) (2 marks) <br>
You can split the dataset into 70% train and 30% test.  

In [18]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)  
y <- data$Sales_Performance

sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

lambda_value <- 1
ridge_model_best <- glmnet(X_train, y_train, alpha = 0, lambda = lambda_value)

predictions_best <- predict(ridge_model_best, s = lambda_value, newx = X_test)
predictions_best <- as.vector(predictions_best)

mse_best <- mean((y_test - predictions_best)^2)
print(paste("Mean Squared Error with lambda = 1: ", mse_best))

ss_total_best <- sum((y_test - mean(y_test))^2)
ss_residual_best <- sum((y_test - predictions_best)^2)
r_squared_best <- 1 - (ss_residual_best / ss_total_best)
print(paste("R-squared with lambda = 1: ", r_squared_best))


[1] "Mean Squared Error with lambda = 1:  1306121.24867492"
[1] "R-squared with lambda = 1:  0.555093223327108"


**2)** Is it possible for you to somehow conduct hyperparameter tuning and find the best lambda value for the Ridge Regression model? (Hint: use the cv.glmnet function)   (1 mark)

In [19]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)  
y <- data$Sales_Performance

sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

cv_model <- cv.glmnet(X_train, y_train, alpha = 0)
best_lambda <- cv_model$lambda.min
print(paste("Best lambda value: ", best_lambda))

ridge_model_best <- glmnet(X_train, y_train, alpha = 0, lambda = best_lambda)

predictions_best <- predict(ridge_model_best, s = best_lambda, newx = X_test)
predictions_best <- as.vector(predictions_best)

mse_best <- mean((y_test - predictions_best)^2)
print(paste("Mean Squared Error with best lambda: ", mse_best))

ss_total_best <- sum((y_test - mean(y_test))^2)
ss_residual_best <- sum((y_test - predictions_best)^2)
r_squared_best <- 1 - (ss_residual_best / ss_total_best)
print(paste("R-squared with best lambda: ", r_squared_best))


[1] "Best lambda value:  94.6320185490095"
[1] "Mean Squared Error with best lambda:  1371168.89079081"
[1] "R-squared with best lambda:  0.560259889633502"


**3)** With the optimal lambda, build the model again and print the coefficients of the various dependent variables. What can you comment about the relationship between lambda and the strength of regularization?        (2 marks)


In [20]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)  
y <- data$Sales_Performance

sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

cv_model <- cv.glmnet(X_train, y_train, alpha = 0)
best_lambda <- cv_model$lambda.min
print(paste("Best lambda value: ", best_lambda))

ridge_model_best <- glmnet(X_train, y_train, alpha = 0, lambda = best_lambda)

coefficients <- coef(ridge_model_best)
print("Coefficients with the best lambda value:")
print(coefficients)

# Make predictions on the test data
predictions_best <- predict(ridge_model_best, s = best_lambda, newx = X_test)
predictions_best <- as.vector(predictions_best)

# Calculate Mean Squared Error (MSE)
mse_best <- mean((y_test - predictions_best)^2)
print(paste("Mean Squared Error with best lambda: ", mse_best))

# Calculate R-squared
ss_total_best <- sum((y_test - mean(y_test))^2)
ss_residual_best <- sum((y_test - predictions_best)^2)
r_squared_best <- 1 - (ss_residual_best / ss_total_best)
print(paste("R-squared with best lambda: ", r_squared_best))


[1] "Best lambda value:  96.9476353599338"
[1] "Coefficients with the best lambda value:"
38 x 1 sparse Matrix of class "dgCMatrix"
                                                    s0
(Intercept)                                  67.078849
Product_Weight                               -0.926877
Energy_EfficiencyEnergy Efficient           -31.142006
Energy_EfficiencyStandard                    30.940826
Visibility_Index                           -288.791413
Product_CategoryCameras                      48.441697
Product_CategoryHeadphones                   25.219096
Product_CategoryHome Theater Systems         99.673030
Product_CategoryKeyboards                    27.641201
Product_CategoryLaptops                     -14.515477
Product_CategoryMicrowaves                   -3.167509
Product_CategoryMiscellaneous Electronics    17.869466
Product_CategoryMonitors                     44.232388
Product_CategoryMouse                       159.468518
Product_CategoryPrinters                   

A larger lambda value typically results in more coefficients being closer to zero, indicating stronger regularization. Conversely, a smaller lambda value will generally yield coefficients that are less penalized and closer to their values without regularization.

## Lasso Regression

Lasso regression is similar to Ridge Regression except that instead of L2 regularization, it employs L1 regularization to address the very same issues that Ridge Regression addresses. 

There are however, a couple of differences between the two. The first and most obvious being that since Lasso Regression implements L1 regularization, the penalty term in this case is proportional to the absolute value of the coefficient. 

Another point to note is that unlike its Ridge counterpart, Lasso Regression can push some coefficients to exactly 0. This effectively drops the feature from the predictive model (Similar to how we drop values through visual analysis). Lasso Regression can thus be used effectively for Feature Selection as well.

**1)** Write code to build a Lasso Regression model similar to how you built the Ridge Regression model. This time incorporate hyperparameter tuning right away. So first print the optimal lambda value.                  (2 marks)

In [22]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)  
y <- data$Sales_Performance

set.seed(123)
sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

cv_model_lasso <- cv.glmnet(X_train, y_train, alpha = 1)
best_lambda_lasso <- cv_model_lasso$lambda.min
print(paste("Best lambda value for Lasso: ", best_lambda_lasso))

lasso_model_best <- glmnet(X_train, y_train, alpha = 1, lambda = best_lambda_lasso)



predictions_lasso <- predict(lasso_model_best, s = best_lambda_lasso, newx = X_test)
predictions_lasso <- as.vector(predictions_lasso)

mse_lasso <- mean((y_test - predictions_lasso)^2)
print(paste("Mean Squared Error with best lambda for Lasso: ", mse_lasso))

ss_total_lasso <- sum((y_test - mean(y_test))^2)
ss_residual_lasso <- sum((y_test - predictions_lasso)^2)
r_squared_lasso <- 1 - (ss_residual_lasso / ss_total_lasso)
print(paste("R-squared with best lambda for Lasso: ", r_squared_lasso))


[1] "Best lambda value for Lasso:  12.3252024588061"
[1] "Mean Squared Error with best lambda for Lasso:  1221337.98582547"
[1] "R-squared with best lambda for Lasso:  0.553954572470503"


**2)** Display the coefficients of all the variables. Do you notice some variables being dropped out? Which ones are they?   (1 mark)

In [23]:
coefficients_lasso <- coef(lasso_model_best)
print("Coefficients with the best lambda value for Lasso:")
print(coefficients_lasso)

[1] "Coefficients with the best lambda value for Lasso:"
38 x 1 sparse Matrix of class "dgCMatrix"
                                                    s0
(Intercept)                                 186.852162
Product_Weight                                .       
Energy_EfficiencyEnergy Efficient           -30.649092
Energy_EfficiencyStandard                     .       
Visibility_Index                           -133.996069
Product_CategoryCameras                       .       
Product_CategoryHeadphones                    3.401345
Product_CategoryHome Theater Systems        171.746246
Product_CategoryKeyboards                     .       
Product_CategoryLaptops                     -13.675686
Product_CategoryMicrowaves                    .       
Product_CategoryMiscellaneous Electronics     .       
Product_CategoryMonitors                     30.255815
Product_CategoryMouse                         .       
Product_CategoryPrinters                      .       
Product_CategoryRefri

all the ones with . have been dropped.

## Elastic Net Regression

Elastic Net regression, an advanced form of linear regression, combines the benefits of L1 (Lasso) and L2 (Ridge) regularization methods. By integrating both penalty terms, Elastic Net overcomes the limitations of each, offering resilience against multicollinearity, aiding feature selection, and preventing overfitting. 

This approach makes Elastic Net a very versatile approach for achieving accurate and efficient models by finding a middle ground between dropping parameters and retaining important predictors. 

**1)** Build your Elastic Net Regression model incorporating all the steps we previously followed for ridge and lasso regression. (Play around with the alpha value and find out how it affects the model)            (2 marks)


In [26]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)  
y <- data$Sales_Performance

sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

alpha_values <- seq(0, 1, by = 0.1)
best_results <- list()

for (alpha in alpha_values) {
  cv_model_en <- cv.glmnet(X_train, y_train, alpha = alpha)
  best_lambda_en <- cv_model_en$lambda.min
  best_results[[as.character(alpha)]] <- list(
    lambda = best_lambda_en,
    model = glmnet(X_train, y_train, alpha = alpha, lambda = best_lambda_en),
    mse = NA,
    r_squared = NA
  )
  
  predictions_en <- predict(best_results[[as.character(alpha)]]$model, s = best_lambda_en, newx = X_test)
  predictions_en <- as.vector(predictions_en)
  
  mse_en <- mean((y_test - predictions_en)^2)
  ss_total_en <- sum((y_test - mean(y_test))^2)
  ss_residual_en <- sum((y_test - predictions_en)^2)
  r_squared_en <- 1 - (ss_residual_en / ss_total_en)
  
  best_results[[as.character(alpha)]]$mse <- mse_en
  best_results[[as.character(alpha)]]$r_squared <- r_squared_en
  
  print(paste("Alpha:", alpha))
  print(paste("Best lambda value for alpha =", alpha, ":", best_lambda_en))
  print(paste("Mean Squared Error for alpha =", alpha, ":", mse_en))
  print(paste("R-squared for alpha =", alpha, ":", r_squared_en))
}

best_alpha <- which.min(sapply(best_results, function(x) x$mse))
best_alpha_value <- as.numeric(names(best_results)[best_alpha])
best_lambda_value <- best_results[[as.character(best_alpha_value)]]$lambda

print(paste("Best alpha value based on MSE:", best_alpha_value))
print(paste("Best lambda value for the best alpha:", best_lambda_value))


[1] "Alpha: 0"
[1] "Best lambda value for alpha = 0 : 97.0854764121905"
[1] "Mean Squared Error for alpha = 0 : 1340282.53351252"
[1] "R-squared for alpha = 0 : 0.544726392943607"
[1] "Alpha: 0.1"
[1] "Best lambda value for alpha = 0.1 : 30.3461612396117"
[1] "Mean Squared Error for alpha = 0.1 : 1334092.08155647"
[1] "R-squared for alpha = 0.1 : 0.546829195390754"
[1] "Alpha: 0.2"
[1] "Best lambda value for alpha = 0.2 : 35.0517841815697"
[1] "Mean Squared Error for alpha = 0.2 : 1331412.98889844"
[1] "R-squared for alpha = 0.2 : 0.547739242449909"
[1] "Alpha: 0.3"
[1] "Best lambda value for alpha = 0.3 : 30.8909305578451"
[1] "Mean Squared Error for alpha = 0.3 : 1330505.45554513"
[1] "R-squared for alpha = 0.3 : 0.54804751773736"
[1] "Alpha: 0.4"
[1] "Best lambda value for alpha = 0.4 : 33.6130882858809"
[1] "Mean Squared Error for alpha = 0.4 : 1329984.35673976"
[1] "R-squared for alpha = 0.4 : 0.548224527081902"
[1] "Alpha: 0.5"
[1] "Best lambda value for alpha = 0.5 : 22.32494113

Alpha = 1: Pure Lasso Regression. It tends to zero out less important features completely.

Alpha = 0: Pure Ridge Regression. It shrinks coefficients but does not set them to zero

0 < Alpha < 1: Mix of Lasso and Ridge. Provides a balance between variable selection (Lasso) and coefficient shrinkage (Ridge).
By adjusting alpha, you can observe how the Elastic Net model shifts between Lasso and Ridge characteristics and how this affects the model performance.

Identify and list the variables that have been dropped in the Elastic Net model. How does this compare to the variables dropped in the Lasso model? <br>
What does that tell you about the number of hyperparameters in Elastic Net Regression compared to the other two models?

In [31]:
library(glmnet)
library(dplyr)
library(tidyr)

data <- data %>%
  mutate(across(c(Energy_Efficiency, Product_Category, Store_ID, Store_Size, Location_Type, Store_Type), as.factor))

data_dummies <- model.matrix(Sales_Performance ~ . - 1 - Product_ID, data = data) %>% as.data.frame()

X <- as.matrix(data_dummies)
y <- data$Sales_Performance

set.seed(123)
sample_indices <- sample(seq_len(nrow(data)), size = 0.7 * nrow(data))
X_train <- X[sample_indices, ]
y_train <- y[sample_indices]
X_test <- X[-sample_indices, ]
y_test <- y[-sample_indices]

alpha_values <- seq(0, 1, by = 0.1)
best_results <- list()

for (alpha in alpha_values) {
  cv_model_en <- cv.glmnet(X_train, y_train, alpha = alpha)
  best_lambda_en <- cv_model_en$lambda.min
  model <- glmnet(X_train, y_train, alpha = alpha, lambda = best_lambda_en)
  predictions_en <- predict(model, s = best_lambda_en, newx = X_test)
  mse_en <- mean((y_test - as.vector(predictions_en))^2)
  best_results[[as.character(alpha)]] <- list(
    lambda = best_lambda_en,
    model = model,
    mse = mse_en
  )
}

# Identify the best alpha based on MSE
best_alpha_index <- which.min(sapply(best_results, function(x) x$mse))
best_alpha_value <- as.numeric(names(best_results)[best_alpha_index])
best_lambda_value <- best_results[[as.character(best_alpha_value)]]$lambda
best_model <- best_results[[as.character(best_alpha_value)]]$model

# Extract coefficients for the best model
coefficients_en <- as.matrix(coef(best_model))
best_dropped_vars <- rownames(coefficients_en)[coefficients_en == 0]

# Display results for the best alpha value
cat("Best alpha value based on MSE:", best_alpha_value, "\n")
cat("Best lambda value for the best alpha:", best_lambda_value, "\n")
cat("Variables dropped in the best Elastic Net model:\n")
print(best_dropped_vars)


Best alpha value based on MSE: 1 
Best lambda value for the best alpha: 16.29319 
Variables dropped in the best Elastic Net model:
 [1] "Product_Weight"                           
 [2] "Energy_EfficiencyStandard"                
 [3] "Product_CategoryCameras"                  
 [4] "Product_CategoryHeadphones"               
 [5] "Product_CategoryKeyboards"                
 [6] "Product_CategoryLaptops"                  
 [7] "Product_CategoryMicrowaves"               
 [8] "Product_CategoryMiscellaneous Electronics"
 [9] "Product_CategoryMouse"                    
[10] "Product_CategoryPrinters"                 
[11] "Product_CategorySmartphones"              
[12] "Product_CategorySpeakers"                 
[13] "Product_CategoryTablets"                  
[14] "Product_CategoryWashing Machines"         
[15] "Store_IDOUT013"                           
[16] "Store_IDOUT017"                           
[17] "Store_IDOUT019"                           
[18] "Store_IDOUT046"               

Elastic Net, which combines L1 (Lasso) and L2 (Ridge) regularization, generally results in fewer columns being dropped from the dataframe compared to Lasso alone. This is because Elastic Net balances feature selection and coefficient shrinkage, allowing the retention of more features. Notably, when the alpha value is set to 0.5, the number of dropped columns is similar to that observed with Lasso, suggesting that certain features contribute minimally to the model's performance.

As the alpha value decreases, Elastic Net behaves more like Ridge (L2) regularization, leading to fewer dropped columns, whereas increasing alpha shifts it closer to Lasso (L1), intensifying feature selection and dropping more columns. This dynamic adjustment highlights how the choice of alpha directly influences the balance between L1 and L2 effects.

Elastic Net's dual hyperparameters—lambda and alpha offer greater flexibility compared to Ridge and Lasso, which only tune lambda. However, this added flexibility comes at the cost of increased complexity.